# Kernel methods - Tutorial
---

**Links to access and run the notebook:**

<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/github/maxjr82/MLinQCbook22-KMs/blob/master/notebooks/case_study_kpca.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/maxjr82/MLinQCbook22-KMs/blob/master/notebooks/case_study_kpca.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

Before start coding we should import some standard python libraries that will be necessary for reading and preproccessing the data.

In [1]:
import requests
import numpy as np
import pandas as pd

from io import BytesIO
from scipy.linalg import eigh

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Load the data set

In this tutorial, we are going to use the **SN2 reactions** data set which was designed to probe the *chemical space* of reactions involving methyl halides and halide anions, i.e. X- + CH3Y -> CH3X + Y-. The dataset includes structures for several smaller molecules (up to six atoms) that can be formed in fragmentation reactions, such as CH3X, HX, CHX or CH2X- as well as geometries for H2, CH2, CH3+ and XY interhalogen compounds, where all possible combinations of X,Y = F, Cl, Br, I are considered. Thus, the SN2 reactions data set spans both the configurational (varying molecular geometries) and compositional spaces.<br><br> For more information about the data set, please check the following paper: Unke, O. T. and Meuwly, M. ["PhysNet: A Neural Network for Predicting Energies, Forces, Dipole Moments and Partial Charges"](https://pubs.acs.org/doi/10.1021/acs.jctc.9b00181), J. Chem. Theory Comput. (2019) 15, 6, 3678–3693.

The SN2 reactions dataset is stored as python dictionary in a compressed numpy binary file (.npz) and it can be loaded directly from the zenodo repository using the commands shown below:

In [2]:
path_to_data = 'https://zenodo.org/record/2605341/files/sn2_reactions.npz?download=1'
response = requests.get(path_to_data)
response.raise_for_status()
data = np.load(BytesIO(response.content))

In [3]:
data = dict(data)
print(data.keys())

dict_keys(['R', 'Q', 'D', 'E', 'F', 'Z', 'N'])


We can now check how many molecules are present in the dataset.

In [4]:
print(f"The number of molecules is {data['N'].shape[0]}.")

The number of molecules is 452709.


In [5]:
atomic_charges = np.unique(data['Z'][data['Z'] != 0])
print(f"List of atomic numbers = {atomic_charges}")

List of atomic numbers = [ 1  6  9 17 35 53]


Let's take a look at some of the molecules present in the SN2 dataset. To this end, we will need to install the **py3Dmol** package, which allows us to visualize 3D molecular structures in a jupyter notebook environment.

In [6]:
! pip install py3Dmol

In [7]:
import py3Dmol

def get_labels(data, index):
    atom_types = {1:'H', 6:'C', 9:'F', 17:'Cl', 35: 'Br', 53: 'I'}
    
    n_atoms = data['N'][index]
    Z = data['Z'][index][:n_atoms]
    labels = np.vectorize(atom_types.get)(Z)
    labels = labels.reshape(-1,1)
    return labels

def view_molecule(data, index, style):
    
    labels = get_labels(data,index)
    n_atoms = len(labels)
    coords = data['R'][index][0:n_atoms,:]
       
    xyz = np.concatenate((labels, coords), axis=1)
    xyz_str = [str(i).strip('[]') for i in xyz]
    geom = str(n_atoms) + '\n' + ' ' + '\n'
    geom += '\n'.join(xyz_str)
    geom = geom.replace("'", "")
    
    for k in style.keys():
        assert k in ('line', 'stick', 'sphere', 'carton')
    
    molview = py3Dmol.view(width=350,height=350)
    molview.addModel(geom,'xyz')
        
    molview.setStyle(style)
    molview.setBackgroundColor('0xeeeeee')
    molview.zoomTo()
    
    return molview

In [8]:
idx = 300
s = {'stick': {'radius': .15}, 'sphere': {'scale': 0.20}}
mol = view_molecule(data, idx, s)
mol.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Molecular descriptor (Coulomb matrix)

The SN2 dataset provides information about the molecular structures in the form of Cartesian coordinates. While this is a standard format to represent molecules in quantum chemistry, it has several drawbacks when used as input in ML models. In general, a common step before applying any ML model is to convert the XYZ coordinates of the molecules into some suitable vector representation that takes into account the symmetry invariances inherent to the chemical structures. The [Coulomb matrix](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.058301) (CM) is one type of molecular descriptor based on the inverse pairwise distances between atoms that (partially) fulfill those symmetry requirements. Thus, in the next step, we are going to convert the XYZ coordinates of each molecule in the SN2 dataset into the CM vector representation. To do that, we will use the [QML](https://www.qmlcode.org/) - Quantum Machine Learning - python toolkit as described in the code blocks below.

In [9]:
! pip install qml

     |████████████████████████████████| 41 kB 346 kB/s 
  Created wheel for qml: filename=qml-0.4.0.27-cp37-cp37m-linux_x86_64.whl size=1049631 sha256=40bb6a3c135285b4f74f58f16de10fb9757c762e34ce3ce8a7f0bc4b4852d3a2
  Stored in directory: /root/.cache/pip/wheels/e4/6f/cc/cc6ed17565977d62aab6730e348c2e9d8c51c559896c467f74
Successfully built qml


In [10]:
from qml.representations import generate_coulomb_matrix

max_atoms = np.max(data['N'])
n_samples = data['N'].shape[0]
descriptor_size = np.int(max_atoms * (max_atoms+1)/2)
X = np.full((n_samples, descriptor_size), np.nan)

for i in range(n_samples):
  charges = data['Z'][i][np.nonzero(data['Z'][i]!= 0)]
  n_atoms = len(charges)
  coords = data['R'][i][:n_atoms,:]
  cm = generate_coulomb_matrix(charges, coords, size=max_atoms, 
                               sorting='row-norm')
  X[i] = cm 

In [11]:
print(f'Number of samples = {X.shape[0]}')
print(f'Number of features = {X.shape[1]}')

Number of samples = 452709
Number of features = 21


Let's take a look at one example of the CM descriptor obtained for the SN2 dataset.

In [12]:
X[1]

array([6874.35714489,  155.23818776, 6874.35714489,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ])

In the example above, it is important to note that the CM vector representation obtained for the second molecule of the dataset has only the three first elements different from zero. This happens because CM is a global descriptor which means that the size of the Coulomb matrix depends on the number of atoms in the molecule, and it is given by $N_{atoms} \cdot (N_{atoms} - 1) / 2$. Therefore, in the case of the molecule number 2, we can conclude that it has only two atoms, as you can see in the visualization below:

In [13]:
idx = 1
s = {'stick': {'radius': .15}, 'sphere': {'scale': 0.20}}
mol = view_molecule(data, idx, s)
mol.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Data preparation

Once we have converted the molecular structures into a reasonable vector representation, we can now proceed with some preprocessing steps of the ML modeling pipeline. First, to avoid a high sparcity in the CM representation of the dataset that may induce the unsupervised model to learn some trivial feature of the data as, for example, the number of atoms in the molecule, we are going to filter the SN2 dataset by selecting the molecules having only five or six atoms, this latter being the maximum number of atoms in the full dataset.

In [14]:
max_n_atoms = np.max(data['N'])
indices = np.where(data['N'] >= max_n_atoms-1)[0]
X_filtered = X[indices]

# Here we also filter the full dataset to get the corresponding chemical 
# properties that will be used later.
data_filtered = dict()
for k in data.keys():
  data_filtered[k] = data[k][indices].copy()

In general, kernel methods can become computationally intractable with increasing the number of instances in the dataset. Thus, we will randomly select a small subsample of the full SN2 dataset in order to avoid computational overhead.

In [15]:
n_samples = X_filtered.shape[0]
n_subsamples = 5000

idx_subsamples = np.random.choice(np.arange(n_samples), size=n_subsamples, replace=False)

train_data = X_filtered[idx_subsamples]

for k in data_filtered.keys():
  data_filtered[k] = data_filtered[k][idx_subsamples]

The final step of our data preparation is to apply a rescaling of all features in the Coulomb matrix descriptors that will be used to train the model. As we could see in one example above, the elements of the CM vector (i.e. the model features) have a very large variation. This characteristic can strongly affect the learning performance of ML algorithms that are based on some distance metric, as for example in kernel methods. For PCA in particular, it is assumed that all features are centered around 0 and have variance in the same order. Hence, we will apply here the standard scaler on the CM features, which is calculated as $Z = (X - \bar{X}) / \sigma_{X}$.

In [16]:
scaler = StandardScaler() # from scikit-learn library
scaler.fit(train_data)
train_data = scaler.transform(train_data)

#scaler = QuantileTransformer(n_quantiles=200,output_distribution='normal')
#scaler.fit(train_data)
#train_data = scaler.transform(train_data)

## Kernel PCA from scratch

In the code block below, you can see an step-by-step implementation of the kernel PCA algorithm that uses the *radial basis function* as similarity measure between pairs of data points.

In [17]:
def kpca(X, gamma=None, n_components=2):
    """
    Implementation of the kPCA algorithm with RBF kernel.

    Args:
        X: Input dataset as NumPy array where the samples are stored as rows (N),
           and the features defined as columns (d).wiki
        gamma: A hyparameter controlling the width of the RBF kernel.
        n_components: The number of components to be returned, defaults to 2.
    """
    # STEP 1: calculate the squared Euclidean distances for every pair of points
    # in the original Nxd dimensional dataset.
    N = X.shape[0]
    A = (X*X).sum(axis=1).reshape((N,1))*np.ones(shape=(1,N))
    B = (X*X).sum(axis=1)*np.ones(shape=(N,1))
    D_squared = A + B - 2 * np.dot(X, X.T)

    # STEP 2: compute the kernel Gram matrix.
    if gamma == None:
      gamma = 1/X.shape[1] 
    
    # To implement a different kernel, check the scikit-learn documentation:
    # https://scikit-learn.org/stable/modules/metrics.html
    K = np.exp(-gamma * D_squared)
    
    # STEP 3: center the symmetric NxN kernel matrix.
    one_n = np.ones((N,N)) / N
    K_centered = K - np.dot(one_n,K) - np.dot(K,one_n) + np.dot(one_n,(np.dot(K,one_n)))
    
    # STEP 4: determine the eigenvalues in descending order with corresponding
    #         eigenvectors from the centered kernel matrix.
    eigvals, eigvecs = eigh(K_centered)
    
    # STEP 5: select the eigenvectors (alphas) corresponding to the
    #         highest eigenvalues.
    eigvals = np.flipud(eigvals)
    eigvecs = np.fliplr(eigvecs)
    X_reduced = eigvecs[:,:n_components]

    return X_reduced

In [18]:
X_kpca = kpca(train_data, gamma=0.05)

The `X_kpca` variable generated by our function contains the information of the input data transformed by the kernel PCA method which compressed the total number of fetures in the original CM dataset (21 features) to only two. Thus, it is now possible to visualize the information contained in the SN2 dataset in a simple 2D scatter plot. Before we do that, let's first group together the coordinates provided by the KPCA model and other chemical properties from the original SN2 dataset into a single table using the pandas library.

In [19]:
df = pd.DataFrame(X_kpca, columns=['KPCA1', 'KPCA2'])
df['Number of atoms'] = data_filtered['N']
df['Halide'] = [', '.join(get_labels(data_filtered, i)[-2:].flatten().tolist()) 
                for i in range(data_filtered['R'].shape[0])]
df['Total charge'] = data_filtered['Q']
df['Total dipole'] = np.linalg.norm(data_filtered['D'], axis=1)
df['Potential energy'] = data_filtered['E']

In [20]:
df

,KPCA1,KPCA2,Number of atoms,Halide,Total charge,Total dipole,Potential energy
0,0.014502,0.011969,6,"F, Cl",-1.0,4.867148,-13.330165
1,-0.015879,0.017417,6,"I, I",-1.0,1.801750,-10.738117
2,0.010788,0.001963,6,"F, I",-1.0,2.051279,-14.363014
3,0.019483,0.004184,6,"F, Br",-1.0,2.294660,-15.147681
4,-0.015564,0.021493,6,"I, I",-1.0,1.727223,-12.971405
...,...,...,...,...,...,...,...
4995,-0.017670,0.022561,6,"Br, I",-1.0,1.591941,-13.232584
4996,0.016682,-0.005839,6,"Cl, Cl",-1.0,8.684119,-12.779014
4997,0.010417,0.001436,6,"F, I",-1.0,1.963044,-14.246324
4998,-0.020270,-0.009149,6,"Br, I",-1.0,2.985360,-11.727135


## Visualizing the KPCA map

In [21]:
! pip install plotly -U

     |████████████████████████████████| 25.3 MB 75.6 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [22]:
import plotly.express as px

fig = px.scatter(df, x="KPCA1", y="KPCA2", color="Potential energy", opacity=0.5, 
                 color_continuous_scale='jet', title="Kernel PCA with RBF", 
                 hover_data={'Potential energy':':.3f', 'Total charge':':.3f', 
                             'Total dipole':':.3f', 'Number of atoms':':.1d',
                             'KPCA1':False, 'KPCA2':False, 
                             'Halide':True})

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Rockwell"
    )
)

fig.update_xaxes(showspikes=True, spikecolor="black", spikethickness=1.5, spikesnap="cursor", spikemode="across")
fig.update_yaxes(showspikes=True, spikecolor="black", spikethickness=1.5)
fig.update_layout(spikedistance=1000, hoverdistance=500)

fig.show()

The nice thing about *dimensionality reduction*, and kernel PCA in particular, is that we can easily identify (by naked eyes) groups of points that share some similar properties even though these properties have never been used explicitly in the training process. In the plot shown above, for instance, we can see that molecules having high to intermediate values of potential energy tends to appear on the left side of the KPCA map, whereas those molecules associated with lower potential energy appear clustered in the right-bottom corner of the plot.